In [10]:
import json, glob, boto3, os
import pdb
import pandas as pd
from pandas.io.json import json_normalize

In [2]:
# from https://alexwlchan.net/2019/07/listing-s3-keys/
def get_matching_s3_objects(bucket, prefix="", suffix=""):
    """
    Generate objects in an S3 bucket.

    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch objects whose key starts with
        this prefix (optional).
    :param suffix: Only fetch objects whose keys end with
        this suffix (optional).
    """
    s3 = boto3.client("s3")
    paginator = s3.get_paginator("list_objects_v2")

    kwargs = {'Bucket': bucket}

    # We can pass the prefix directly to the S3 API.  If the user has passed
    # a tuple or list of prefixes, we go through them one by one.
    if isinstance(prefix, str):
        prefixes = (prefix, )
    else:
        prefixes = prefix

    for key_prefix in prefixes:
        kwargs["Prefix"] = key_prefix

        for page in paginator.paginate(**kwargs):
            try:
                contents = page["Contents"]
            except KeyError:
                return

            for obj in contents:
                key = obj["Key"]
                if key.endswith(suffix):
                    yield obj


def get_matching_s3_keys(bucket, prefix="", suffix=""):
    """
    Generate the keys in an S3 bucket.

    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch keys that start with this prefix (optional).
    :param suffix: Only fetch keys that end with this suffix (optional).
    """
    for obj in get_matching_s3_objects(bucket, prefix, suffix):
        yield obj["Key"]

In [3]:
session = boto3.Session()
BUCKET_NAME = 'snowbot-pv'

# S3 Connect
s3 = session.resource('s3')

bucket = s3.Bucket(BUCKET_NAME)

In [7]:
DATA_DIR = "./data/"
MERGED_FILENAME = "merged_file.json"
merged_file = DATA_DIR + MERGED_FILENAME

result = []

for f in get_matching_s3_keys(BUCKET_NAME, suffix="lifts.json"):
    
    # Write the file from S3 into a local temp file
    with open('temp', 'wb') as tfw:
        bucket.download_fileobj(f, tfw)

    # Append the local temp file into the result list
    with open('temp', 'rb') as tfr:          
        result.append(json.load(tfr))
        
os.remove("temp")

# Fill the output file with the merged content
with open(merged_file, "w") as outfile:
     json.dump(result, outfile)

In [11]:
#load the merged json as a dataframe
with open(merged_file, "r") as f:
    d = json.load(f)
    df = pd.DataFrame.from_dict(json_normalize(d, record_path='lifts', meta='timestamp'))

In [12]:
# set datatypes
df = df.astype({
    "liftID": 'category',
    "resortID": 'category',
    "liftName": 'category',
    "status": 'category',
    "timeToRide": "int"
})
df["timestamp"] = pd.to_datetime(df["timestamp"])

In [13]:
def get_status_changes(df):
    '''Returns a dataframe that only includes the times when there was a change to a new status'''
    df = df.groupby('liftName', group_keys=False)\
           .apply(lambda x: x[x.status.ne(x.status.shift())])\
           .reset_index(drop=True)
    return df

In [14]:
get_status_changes(df)

,liftID,resortID,liftName,status,timeToRide,timestamp
0,3,13,7th Heaven Express,O,6,2020-01-04 14:19:09.425451-08:00
1,3,13,7th Heaven Express,X,6,2020-01-04 14:49:08.722630-08:00
2,3,13,7th Heaven Express,O,6,2020-01-05 12:49:10.222813-08:00
3,3,13,7th Heaven Express,X,6,2020-01-05 14:49:08.549014-08:00
4,36,13,Big Red Express,H,8,2020-01-03 07:49:08.848262-08:00
...,...,...,...,...,...,...
164,72,13,Whistler Village Gondola Upper,O,11,2020-01-04 13:19:08.977980-08:00
165,72,13,Whistler Village Gondola Upper,X,11,2020-01-04 15:19:08.879936-08:00
166,72,13,Whistler Village Gondola Upper,H,11,2020-01-05 07:49:08.680133-08:00
167,72,13,Whistler Village Gondola Upper,O,11,2020-01-05 08:19:08.762177-08:00


In [142]:
# add:
# 
# daily: for each chair calculate most open status of the day: O > H > X
# Days since each chair was last seen open with timestamp of most recent open time.
# snowfall since last open
# save data for other mountains